In [1]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.mixture import DPGMM
from tabulate import tabulate

In [2]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file
from functions import *

In [3]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)



4885

In [4]:
num_upcoming_fights = len(df_upcoming)

In [5]:
df = df_upcoming.append(df)
len(df)

4897

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
def return_finish_type(winner, finish):
    #print(winner, finish)
    #Why overcomplicate things?  We can just use a few if statements
    if winner == 'Red':
        #print("HI")
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Red - DEC')
        if finish in ['SUB']:
            return('Red - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Red - KO/TKO')
    if winner == 'Blue':
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Blue - DEC')
        if finish in ['SUB']:
            return('Blue - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Blue - KO/TKO')
        
    #Test for NaN
    if finish != finish:
        return('')
    
    if finish == 'Overturned':
        return('')
    
    
    return ('error')

In [8]:
#This calls for the power of lambda!
df['finish_type'] = df.apply(lambda x: return_finish_type(x['Winner'], x['finish']), axis=1)

In [9]:
finish_list = ['Red - DEC', 'Red - SUB', 'Red - KO/TKO', 'Blue - DEC', 'Blue - SUB', 'Blue - KO/TKO']

#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.

for f in range(len(finish_list)):
    mask = df['finish_type'] == finish_list[f]
    df['label'][mask] = f
    
#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-af8f7542de83>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = f


In [10]:
df.rename(columns={'r_dec_odds': 'Red - DEC', 'r_sub_odds': 'Red - SUB', 'r_ko_odds': 'Red - KO/TKO',
                'b_dec_odds': 'Blue - DEC', 'b_sub_odds': 'Blue - SUB', 'b_ko_odds': 'Blue - KO/TKO'}, inplace=True)

In [11]:
df_label = df['label']
df_odds = df[finish_list]

In [12]:
df_train = df[num_upcoming_fights:]
odds_train = df_odds[num_upcoming_fights:]
label_train = df_label[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = df_odds[:num_upcoming_fights]
label_test = df_label[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

12
12
12
4885
4885
4885


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round_time,total_fight_time_secs,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Alexander Volkanovski,Brian Ortega,-180.0,155,55.555556,155.0,2021-09-25,"Las Vegas, Nevada, USA",USA,Red,...,5:00,1500.0,140.0,500.0,1600.0,400.0,250.0,700.0,Red - DEC,0.0
1,Valentina Shevchenko,Lauren Murphy,-1375.0,800,7.272727,800.0,2021-09-25,"Las Vegas, Nevada, USA",USA,Red,...,4:00,1140.0,200.0,1800.0,350.0,2800.0,-110.0,2500.0,Red - KO/TKO,2.0
2,Nick Diaz,Robbie Lawler,140.0,-160,140.000000,62.5,2021-09-25,"Las Vegas, Nevada, USA",USA,Blue,...,0:44,644.0,275.0,225.0,800.0,2500.0,650.0,185.0,Blue - KO/TKO,5.0
3,Curtis Blaydes,Jairzinho Rozenstruik,-380.0,290,26.315789,290.0,2021-09-25,"Las Vegas, Nevada, USA",USA,Red,...,5:00,900.0,180.0,1000.0,1000.0,3500.0,125.0,380.0,Red - DEC,0.0
4,Jessica Andrade,Cynthia Calvillo,-220.0,180,45.454545,180.0,2021-09-25,"Las Vegas, Nevada, USA",USA,Red,...,4:54,294.0,140.0,330.0,1000.0,900.0,225.0,1400.0,Red - KO/TKO,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4880,Duane Ludwig,Darren Elkins,-155.0,135,64.516129,135.0,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0:44,44.0,NaN,NaN,NaN,NaN,NaN,NaN,Blue - KO/TKO,5.0
4881,John Howard,Daniel Roberts,-210.0,175,47.619048,175.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2:01,121.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4882,Brendan Schaub,Chase Gormley,-260.0,220,38.461538,220.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0:47,47.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4883,Mike Pierce,Julio Paulino,-420.0,335,23.809524,335.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - DEC,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round_time,total_fight_time_secs,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Thiago Santos,Johnny Walker,-145.0,125,68.965517,125.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,800.0,900.0,2000.0,1600.0,-110.0,175.0,,NaN
1,Kevin Holland,Kyle Daukaus,-170.0,150,58.823529,150.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,200.0,250.0,1100.0,700.0,215.0,1600.0,,NaN
2,Alex Oliveira,Niko Price,165.0,-195,165.000000,51.282051,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,450.0,350.0,700.0,1100.0,550.0,120.0,,NaN
3,Misha Cirkunov,Krzysztof Jotko,115.0,-135,115.000000,74.074074,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,550.0,275.0,275.0,1400.0,600.0,185.0,,NaN
4,Alexander Hernandez,Mike Breeden,-630.0,450,15.873016,450.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,175.0,900.0,500.0,3500.0,110.0,1100.0,,NaN
5,Joe Solecki,Jared Gordon,-135.0,115,74.074074,115.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,165.0,200.0,400.0,1200.0,900.0,600.0,,NaN
6,Antonina Shevchenko,Casey O'Neill,190.0,-235,190.000000,42.553191,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,300.0,200.0,1600.0,330.0,1000.0,300.0,,NaN
7,Bethe Correia,Karol Rosa,435.0,-600,435.000000,16.666667,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,700.0,-200.0,2500.0,600.0,1600.0,400.0,,NaN
8,Devonte Smith,Jamie Mullarkey,-145.0,125,68.965517,125.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,330.0,450.0,2000.0,900.0,140.0,330.0,,NaN
9,Douglas Silva de Andrade,Gaetano Pirrello,-265.0,215,37.735849,215.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,165.0,450.0,1400.0,2000.0,150.0,600.0,,NaN


In [15]:
with open('../data/production_models_mov.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)

###REVERT THIS
#model_choice = 0

###


model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
The model choice is 0
[['model_2021_07_15'], ['LinearDiscriminantAnalysis()'], ['["R_Women\'s Featherweight_rank", \'B_Lightweight_rank\', \'R_Featherweight_rank\', \'R_Light Heavyweight_rank\', \'total_title_bout_dif\', \'R_losses\', \'B_win_by_Submission\', \'Blue - DEC\', \'B_draw\', \'R_Middleweight_rank\', \'B_avg_SIG_STR_pct\', "B_Women\'s Strawweight_rank", \'age_dif\', \'B_Middleweight_rank\', \'avg_sub_att_dif\', \'avg_td_dif\', \'B_avg_SUB_ATT\', "R_Women\'s Flyweight_rank", \'R_Pound-for-Pound_rank\', \'R_Reach_cms\', \'R_wins\', \'B_ev\', \'R_Weight_lbs\', \'sub_dif\', \'R_ev\', \'B_avg_TD_landed\']'], ['0'], ['10235.0']]

LinearDiscriminantAnalysis() ["R_Women's Featherweight_rank", 'B_Lightweight_rank', 'R_Featherweight_rank', 'R_Light Heavyweight_rank', 'total_title_bout_dif', 'R_losses', 'B_win_by_Submission', 'Blue - DEC', 'B_draw', 'R_Middleweight_rank', 'B_avg_SIG_STR_pct', "B_Women's Strawweight_rank", 'age_dif', 'B_Middleweight_rank', 'avg_sub_att_dif', 'avg_td_d

In [16]:
def display_bets(df, min_ev, finish_list, save_to_html=False):
    bet_list = []
    print("min_ev = " + str(min_ev))
    for index, row in df.iterrows():
        print()
        print()
        print(f"{row['t1_name']} (RED) vs. {row['t2_name']} (BLUE)")
        for f in finish_list:
            ev = get_bet_ev(row[f], row[f+'_prob'])
            if ev > min_ev:
                print(f"BET: {f}. ODDS: {row[f]}. PROB: {row[f+'_prob']}.  EV: {ev}")
                bet_list.append([row['t1_name'], row['t2_name'], f, row[f]])
                
    #print(bet_list)
    export_bets = pd.DataFrame(bet_list, columns=['Red', 'Blue', 'Bet', 'Odds'])
    if save_to_html:
        export_bets.to_csv('bets.csv', index=False)

In [17]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, finish_list, verbose=True, save_to_html=False):
    fighters_test = df[['R_fighter', 'B_fighter']]
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    #print(labels_train)
    print(len(df_train))
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    
    labels_train = labels_train.dropna()
    odds_train = odds_train[odds_train.index.isin(labels_train.index)]     
    df_train = df_train[df_train.index.isin(labels_train.index)] 

    
    
    #input_model.fit_transform(df_train, labels_train)
    #probs = input_model.predict_proba(df_test)
    
    
    
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(df_train)

    input_model.fit(scaled_train, labels_train)

    #print(scaled_train)
    
    scaled_test = scaler.transform(df_test)

    probs = input_model.predict_proba(scaled_test)    

    
    
    
    
    
    
    
    
    
    prob_label_list = []
    for f in finish_list:
        prob_label_list.append(f + '_prob')
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #print(probs)
    #print()
    #print()
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, 0], odds_test[:, 1],
                            odds_test[:, 2], odds_test[:, 3],odds_test[:, 4], odds_test[:, 5],
                            probs[:, 0], probs[:, 1],probs[:, 2], probs[:, 3],probs[:, 4], probs[:, 5]))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name'] + finish_list + prob_label_list)

    display_bets(ev_prepped_df, input_ev, finish_list, save_to_html=save_to_html)
    
    return(ev_prepped_df)

In [18]:
output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, finish_list, verbose = True, save_to_html=True)

3455
min_ev = 0


Thiago Santos (RED) vs. Johnny Walker (BLUE)
BET: Red - DEC. ODDS: 800.0. PROB: 0.2574295036041183.  EV: 131.6865532437065
BET: Blue - SUB. ODDS: 1600.0. PROB: 0.17401697490321402.  EV: 195.82885733546385


Kevin Holland (RED) vs. Kyle Daukaus (BLUE)
BET: Red - SUB. ODDS: 1100.0. PROB: 0.12751127155266664.  EV: 53.01352586319996
BET: Blue - KO/TKO. ODDS: 1600.0. PROB: 0.1473511401667529.  EV: 150.49693828347995


Alex Oliveira (RED) vs. Niko Price (BLUE)
BET: Red - DEC. ODDS: 450.0. PROB: 0.2541008430978803.  EV: 39.75546370383415
BET: Blue - SUB. ODDS: 1100.0. PROB: 0.17466022048948762.  EV: 109.59226458738513


Misha Cirkunov (RED) vs. Krzysztof Jotko (BLUE)
BET: Red - DEC. ODDS: 550.0. PROB: 0.17639112150103195.  EV: 14.654228975670776
BET: Red - SUB. ODDS: 275.0. PROB: 0.4197589522515091.  EV: 57.409607094315916


Alexander Hernandez (RED) vs. Mike Breeden (BLUE)
BET: Blue - KO/TKO. ODDS: 1100.0. PROB: 0.08822557727324988.  EV: 5.870692727899865


Joe Solecki (RED

In [19]:
output_df

,t1_name,t2_name,Red - DEC,Red - SUB,Red - KO/TKO,Blue - DEC,Blue - SUB,Blue - KO/TKO,Red - DEC_prob,Red - SUB_prob,Red - KO/TKO_prob,Blue - DEC_prob,Blue - SUB_prob,Blue - KO/TKO_prob
0,Thiago Santos,Johnny Walker,800.0,2000.0,-110.0,900.0,1600.0,175.0,0.257430,0.037280,0.313721,0.059708,0.174017,0.157845
1,Kevin Holland,Kyle Daukaus,200.0,1100.0,215.0,250.0,700.0,1600.0,0.290881,0.127511,0.206543,0.172596,0.055117,0.147351
2,Alex Oliveira,Niko Price,450.0,700.0,550.0,350.0,1100.0,120.0,0.254101,0.062611,0.115663,0.194344,0.174660,0.198621
3,Misha Cirkunov,Krzysztof Jotko,550.0,275.0,600.0,275.0,1400.0,185.0,0.176391,0.419759,0.089948,0.150254,0.019719,0.143929
4,Alexander Hernandez,Mike Breeden,175.0,500.0,110.0,900.0,3500.0,1100.0,0.346394,0.146667,0.315400,0.086391,0.016922,0.088226
5,Joe Solecki,Jared Gordon,165.0,400.0,900.0,200.0,1200.0,600.0,0.305502,0.146849,0.109970,0.314193,0.014448,0.109037
6,Antonina Shevchenko,Casey O'Neill,300.0,1600.0,1000.0,200.0,330.0,300.0,0.172465,0.055139,0.078779,0.225900,0.393618,0.074099
7,Bethe Correia,Karol Rosa,700.0,2500.0,1600.0,-200.0,600.0,400.0,0.090592,0.016374,0.030716,0.548237,0.082044,0.232036
8,Devonte Smith,Jamie Mullarkey,330.0,2000.0,140.0,450.0,900.0,330.0,0.285003,0.087754,0.214950,0.237049,0.066640,0.108604
9,Douglas Silva de Andrade,Gaetano Pirrello,165.0,1400.0,150.0,450.0,2000.0,600.0,0.469029,0.066290,0.165816,0.174228,0.027882,0.096754
